In [34]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library import EfficientSU2, TwoLocal, ZZFeatureMap
from qiskit.visualization import plot_histogram, plot_distribution
from qiskit.result import QuasiDistribution
from qiskit.transpiler.passes import Decompose
from qiskit.quantum_info import Operator

from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.algorithms import MinimumEigenOptimizer

from qiskit_algorithms import VQE, NumPyMinimumEigensolver, SamplingVQE
from qiskit_algorithms.optimizers import COBYLA, GradientDescent, Optimizer, Minimizer
from qiskit_algorithms.gradients import  ParamShiftEstimatorGradient #, FiniteDiffEstimatorGradient, LinCombEstimatorGradient, QFI, DerivativeType, LinCombQGT

from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider, YahooDataProvider

from qiskit.primitives import BackendSampler, StatevectorEstimator, StatevectorSampler
from qiskit.primitives import Estimator
from qiskit_aer.primitives import Sampler

#Aer Sim

from scipy.optimize import minimize
from sklearn.preprocessing import normalize
from docplex.mp.model import Model
import numpy as np
import matplotlib.pyplot as plt
import datetime
import warnings

In [23]:
#Necessary for tracking convergence
estimator = Estimator()
sampler = StatevectorSampler()
optimizer = COBYLA()
optimizer.set_options(maxiter=500)
converter = QuadraticProgramToQubo()

# Gradient decsent methods
#grad_finite = FiniteDiffEstimatorGradient(estimator, epsilon=0.001)
#grad_SPSA = SPSAEstimatorGradient(estimator, epsilon=0.001, batch_size=10, seed=50) #import SPSAEstimatorGradient
#qgt = LinCombQGT(estimator, derivative_type=DerivativeType.COMPLEX)
#pshift_sampler = ParamShiftSamplerGradient(sampler)
#grad_natural = QFI(qgt)
grad_pshift = ParamShiftEstimatorGradient(estimator) # import , ParamShiftSamplerGradient
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [4]:
# set number of assets which is equal to number of qubits
num_assets = 25
seed = 479

# Generate expected return and covariance matrix from (random) time-series
stocks = [("TICKER%s" % i) for i in range(num_assets)]
data = RandomDataProvider( # Change to YahooDataProvider for real stock data
    tickers=stocks,
    start=datetime.datetime(2023, 1, 1),
    end=datetime.datetime(2023, 1, 30),
    seed=seed,
)

data.run()
ev = data.get_period_return_mean_vector() # Initial state to optimize over
covariance = data.get_period_return_covariance_matrix() # Hamiltonian Matrix

risk_factor = 0.5  # set risk factor
budget = 20  # set budget
#penalty = num_assets  # set parameter to scale the budget penalty term

qp = PortfolioOptimization(expected_returns=ev, covariances=covariance, risk_factor=risk_factor, budget=budget).to_quadratic_program()
qubo = converter.convert(qp) #constraintless format to pass as operator

In [63]:
# 3 common quantum circuits to use for ansatz with linear entanglement
ZZ_linear = ZZFeatureMap(feature_dimension=num_assets, entanglement='linear', reps=1)
SU2_linear = EfficientSU2(num_assets, entanglement='linear', reps=1)
TwoL_linear = TwoLocal(num_assets, 'ry', 'cx', entanglement='linear', reps=1)


# 3 common quantum circuits to use for ansatz with cyclic entanglement
ZZ_cyclic = ZZFeatureMap(feature_dimension=num_assets, entanglement='circular', reps=1) 
SU2_cyclic = EfficientSU2(num_assets, entanglement='circular', reps=1)
TwoL_cyclic = TwoLocal(num_assets, 'ry', 'cx', entanglement='circular', reps=1)


# 3 common quantum circuits to use for ansatz with full entanglement
ZZ_full = ZZFeatureMap(feature_dimension=num_assets, entanglement='full', reps=1)
SU2_full = EfficientSU2(num_assets, entanglement='full', reps=1)
TwoL_full = TwoLocal(num_assets, 'ry', 'cx', entanglement='full', reps=1)


# master list for ansatz circuits 
ansatz_list = [[ZZ_linear, ZZ_full],
                [SU2_linear, SU2_cyclic, SU2_full],
                [TwoL_linear, TwoL_cyclic, TwoL_full]]


# This starts the optimization considering all assets with equal weight
ZZ_params = [0.1] * 25    # ZZFeatureMap circuit has 25 parameters,
SU2_params = [0.1] * 100   # EfficentSU2 circuit has 100 parameters
TwoL_params = [0.1] * 50 # TwoLocal circuit has 50 parameters

print(TwoL_full.num_parameters)

50


In [64]:
step = 0.1 # Naive step amount
H, vec = qubo.to_ising()

In [65]:
# Standard Gradient descent method
def gd(init_params, iterations, H, step, qc_list):
    params = [[init_params], [init_params], [init_params]]
    for qc in qc_list:
        j = 0
        for i in range(iterations):
            grad = grad_pshift.run([qc], [H], [params[j][i]]).result().gradients[0]
            params[j].append(params[j][i] - step * grad)      
        j += 1
        #step = step * .75 # Modification to decay the step distance 
    return params

def estimate_energies(p, params, H, qc):
    if qc == "ZZ":
        qc = ZZFeatureMap(feature_dimension=num_assets, entanglement='linear', reps=1)
    elif qc == "SU2":
        qc = EfficientSU2(num_assets, entanglement='linear', reps=1)
    else:
        qc = TwoLocal(num_assets, 'ry', 'cx', entanglement='linear', reps=1)
    return [[estimator.run([qc], [H], pair).result().values[0] for pair in pi] for pi in params]

def plot_energies(p, params, H, qc):
    if qc == "ZZ": 
        e = estimate_energies(p, params, H, "ZZ")
    elif qc == "SU2":
        e = estimate_energies(p, params, H, "SU2")
    else:
        e = estimate_energies(p, params, H, "TwoL")
    n = len(e[0])
    labels = ['linear', 'cyclic', 'full']
    for i, s in enumerate(e):
        plt.scatter(np.arange(n), s, label=labels[i])
    plt.title(f'Gradient Analysis for $H_1$: $p=${p}, Naive $\mu={step}$')
    plt.legend(loc=1)
    plt.ylabel('Estimated Energy')
    plt.xlabel('Iteration')
    plt.show()

In [69]:
### Only uncomment when ready to analyze convergence ###
## This cell takes roughly 6 hours to run on an M2 Pro Chipped Mac for reference
#ZZ_result = gd(ZZ_params, 25, H, step, ansatz_list[0])
#plot_energies(1, ZZ_result, H, "ZZ")

In [70]:
### Only uncomment when ready to analyze convergence ###
## This cell takes roughly 6 hours to run on an M2 Pro Chipped Mac for reference
#SU2_result = gd(SU2_params, 25, H, step, ansatz_list[1])
#plot_energies(1, SU2_result, H, "SU2")

In [71]:
### Only uncomment when ready to analyze convergence ###
## This cell takes roughly 6 hours to run on an M2 Pro Chipped Mac for reference
#TwoL_result = gd(TwoL_params, 25, H, step, ansatz_list[2])
#plot_energies(1, TwoL_result, H, "TwoL")

In [29]:
best_ansatz = ZZ_linear #placeholder till we know which one is the best circuit for this task
svqe = MinimumEigenOptimizer(SamplingVQE(sampler=Sampler(), ansatz=best_ansatz, optimizer=optimizer))
result = svqe.solve(qp)

In [67]:
def print_result(result):
    selection = result.x
    value = result.fval
    print("Optimal: selection {}, value {:.4f}".format(selection, value))

    eigenstate = result.min_eigen_solver_result.eigenstate
    probabilities = (
        eigenstate.binary_probabilities()
        if isinstance(eigenstate, QuasiDistribution)
        else {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}
    )
    print("\n----------------- Full result ---------------------")
    print("This is a list of all possible porfolio options and their valuation:")
    print("\n\t\t selection\t\t\t\t value\t\tprobability")
    print("---------------------------------------------------")
    probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    for k, v in probabilities:
        x = np.array([int(i) for i in list(reversed(k))])
        value = qp.objective.evaluate(x)
        print("%10s\t%.4f\t\t%.4f" % (x, value, v))


print_result(result)

Optimal: selection [1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 0.], value 0.2353

----------------- Full result ---------------------
This is a list of all possible porfolio options and their valuation:

		 selection				 value		probability
---------------------------------------------------
[1 1 0 1 1 1 1 1 0 1 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0]	0.1594		0.0010
[1 0 0 0 1 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 1 1]	0.0600		0.0010
[1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1]	0.1452		0.0010
[1 0 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 1 0 0 0 1 0 0 0]	0.0130		0.0010
[1 0 1 0 0 1 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 1 0 0]	0.0973		0.0010
[1 0 1 0 1 1 0 1 1 0 1 1 0 1 1 0 0 1 1 1 0 0 1 1 0]	0.1323		0.0010
[1 0 0 1 0 1 1 1 1 1 0 1 1 0 0 1 0 0 0 0 0 1 1 1 1]	0.0849		0.0010
[0 1 1 0 0 1 0 0 1 0 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0]	0.0095		0.0010
[0 0 1 1 0 1 0 0 1 1 0 1 1 1 0 1 0 1 1 0 1 1 1 0 0]	0.0803		0.0010
[1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 0 1 1]	0.1466		0.0010
[1 0 0 0